# Estimación de dirección de arribo de fuentes sonoras 

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from tdoa_py import correlation, simulation_room

## Carga de audios anecoicos

In [19]:
angle, wavs = simulation_room.sim_room_Nmics(
    wav_path = 'files/anechoic/Flamenco1_U89.wav',
    out_dir = 'files/pyroom_mod',
    fs=48000,

    room_dim=(100, 100, 100),
    rt60=3,
    snr_db=70,

    n_mics=4,
    mic_d=0.1,
    mic_z=1.2,
    mic_directivity='omni',

    src_dist=50.0,
    src_az_deg=45.0,
    src_z=1.2
)

Fuente en: x=85.36, y=85.36, z=1.20


In [ ]:

avg_angle_deg, avg_tdoa, hemi_avgs = estimate_doa_from_wavs(
    wav_folder = r'C:\Users\faust\Desktop\DSP\Trabajo Práctico\Audios Room',
    mic_d = 0.1,
    fs = 44100
)